In [1]:
# this is a test to see if GitHub is derping
import pandas as pd
df = pd.read_csv('heart_disease_uci(1).csv')

df.head

<bound method NDFrame.head of      age     sex        dataset               cp  trestbps   chol    fbs  \
0     63    Male      Cleveland   typical angina     145.0  233.0   True   
1     67    Male      Cleveland     asymptomatic     160.0  286.0  False   
2     67    Male      Cleveland     asymptomatic     120.0  229.0  False   
3     37    Male      Cleveland      non-anginal     130.0  250.0  False   
4     41  Female      Cleveland  atypical angina     130.0  204.0  False   
..   ...     ...            ...              ...       ...    ...    ...   
915   54  Female  VA Long Beach     asymptomatic     127.0  333.0   True   
916   62    Male  VA Long Beach   typical angina       NaN  139.0  False   
917   55    Male  VA Long Beach     asymptomatic     122.0  223.0   True   
918   58    Male  VA Long Beach     asymptomatic       NaN  385.0   True   
919   62    Male  VA Long Beach  atypical angina     120.0  254.0  False   

              restecg  thalch  exang  oldpeak        slop

In [2]:
# Heart Disease Dataset Modeling

# ------------------------------
# 1. Import Libraries
# ------------------------------
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

# ------------------------------
# 2. Load and Preprocess Data
# ------------------------------
df = pd.read_csv(r"C:\Users\aryan\machinel\Assignment1\Data\heart_disease_uci(1).csv")  # Adjust filename if needed

# Convert types if needed
df['sex'] = df['sex'].astype(str)
df['fbs'] = df['fbs'].astype(str)
df['exang'] = df['exang'].astype(str)

# Encode target
df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)

# One-hot encoding
df = pd.get_dummies(df, drop_first=True)

# Clean column names
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df.columns = df.columns.str.replace('(', '')
df.columns = df.columns.str.replace(')', '')

# ------------------------------
# 3. Regression (ElasticNet)
# ------------------------------
X_reg = df.drop(columns=['num', 'target'])
y_reg = df['num']

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

model = ElasticNet()
model.fit(X_train_reg, y_train_reg)

# Predict & Evaluate
y_pred = model.predict(X_test_reg)
mse = mean_squared_error(y_test_reg, y_pred)
r2 = r2_score(y_test_reg, y_pred)

print("Regression Results (ElasticNet):")
print(f"Mean Squared Error: {mse:.3f}")
print(f"R² Score: {r2:.3f}\n")

# ------------------------------
# 4. Classification - Logistic Regression
# ------------------------------
X_class = df.drop(columns=['num', 'target'])
y_class = df['target']

X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_log, y_train_log)

# Predict & Evaluate
y_pred_log = logreg.predict(X_test_log)
print("Classification Results (Logistic Regression):")
print("Accuracy:", accuracy_score(y_test_log, y_pred_log))
print("\nClassification Report:\n", classification_report(y_test_log, y_pred_log))
ConfusionMatrixDisplay.from_predictions(y_test_log, y_pred_log)
plt.title("Confusion Matrix: Logistic Regression")
plt.show()

# ------------------------------
# 5. Classification - k-NN
# ------------------------------
X_train, X_test, y_train, y_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)
print("Classification Results (k-NN):")
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("\nClassification Report:\n", classification_report(y_test, y_pred_knn))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_knn)
plt.title("Confusion Matrix: k-NN")
plt.show()

# ------------------------------
# 6. Conclusion (Markdown)
# ------------------------------
# Include the conclusion in a markdown cell in your notebook:
#
# ## Conclusion
# - ElasticNet achieved an R^2 of 0.399 and MSE of 0.146
# - Logistic Regression achieved 91.7% accuracy
# - k-NN achieved 70.0% accuracy
# - Logistic regression outperformed k-NN in all classification metrics
# - Logistic Regression is a better fit for this dataset
 # Verify data loaded

ValueError: Input X contains NaN.
ElasticNet does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values